In [ ]:
#@title DeFooocus (мой личный билд)
#@markdown Запуск DeFoоcus | Подключаемся к T4 среде
#@markdown ****
#@markdown Увага! При работе в интерфейсе с контроллерами FaceSwap и CPDS возможны сбои; При работе с контроллерами ImagePrompt и PyraCanny в 85% случаев работа будет стабильной. Ошибка при остановке выполнения - это норма, не переживаем
#@markdown ****
#@markdown Настройка аргументов запуска (тема, пресет настроек и загружаемых моделей, дополнительные аргументы)
import time
print("[DeFooocus] Подготовка среды выполнения ...")

theme = "dark" #@param ["dark", "light"]
preset = "deafult" #@param ["deafult", "realistic", "anime", "lcm", "sai", "turbo", "lighting", "hypersd", "playground_v2.5", "dpo", "spo", "sd1.5"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16" #@param {type: "string"}

if preset != "deafult":
  args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
  args = f"{advenced_args} --theme {theme}"

!pip install -q pygit2==1.12.2
%cd /content
!git clone https://github.com/ehristoforu/DeFooocus.git
print("Время остановить выполнение этого блока, файловая структура создана, можно приступать к загрузке")
for i in range(10, 0, -1):
    print(f"Обратный отсчёт: {i} секунд")
    time.sleep(1)
%cd /content/DeFooocus
print("[DeFooocus] Установка необходимых компонентов ...")
!pip install -q -r requirements_versions.txt

print("[DeFooocus] Запуск ...")
!python entry_with_update.py $args

[DeFooocus] Подготовка среды выполнения ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 11.6 MB/s eta 0:00:00
/content
Cloning into 'DeFooocus'...
remote: Enumerating objects: 6989, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 6989 (delta 69), reused 116 (delta 68), pack-reused 6867
Receiving objects: 100% (6989/6989), 47.28 MiB | 35.33 MiB/s, done.
Resolving deltas: 100% (3833/3833), done.
Время остановить выполнение этого блока, файловая структура создана, можно приступать к загрузке
Обратный отсчёт: 10 секунд
Обратный отсчёт: 9 секунд
Обратный отсчёт: 8 секунд
Обратный отсчёт: 7 секунд
Обратный отсчёт: 6 секунд
Обратный отсчёт: 5 секунд
Обратный отсчёт: 4 секунд
Обратный отсчёт: 3 секунд
Обратный отсчёт: 2 секунд
Обратный отсчёт: 1 секунд
/content/DeFooocus
[DeFooocus] Установка необходимых компонентов ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━

#Примечание
По достижении надписи "Время остановить выполенение этого блока, файловая структура создана, можно приступать к загрузке", следует повторно нажать на индикатор выполнения кода (белый кружок в левом верхнем углу блока). После загрузки спокойно запускаем первый блок снова, а на ошибки внимание не обрашаем.

In [ ]:
#@title DeFooocus (Base model)
# @title  { display-mode: "form" }
#@markdown ### Загрузка основной модели в civitai (Берем version_id любой модельки с сайта):
model_id = '641087' # @param {type:"string"}
from IPython.display import clear_output
import requests
import os
from tqdm import tqdm
import http.client

http.client.HTTPConnection._http_vsn = 11
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.1'

# API для Civitai
api_key = "37886f12c162b410d63075416e9e0d00"
auth_header = {"Authorization": f"Bearer {api_key}"}
models_url = "https://civitai.com/api/v1/models"
response = requests.get(models_url, headers=auth_header)

if response.status_code == 200:
    print("Успешно авторизованы в civitai.")
    data = response.json()

# URL для загрузки
download_url = f"https://civitai.com/api/download/models/{model_id}"

save_path = "/content/DeFooocus/models/checkpoints/realism.safetensors"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

try:
    with requests.get(download_url, headers=auth_header, stream=True) as response:
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk: # фильтр для держания живых соединений
                    f.write(chunk)
                    progress_bar.update(len(chunk))
        progress_bar.close()
    print(f"Модель {model_id} успешно загружена и сохранена в {save_path}")
except requests.exceptions.RequestException as e:
    print(f"Произошла ошибка при загрузке модели {model_id}: {e}")

#clear_output()
#print('\033[1;32mОхуеть тебе повезло, друг')

Успешно авторизованы в civitai.


100%|██████████| 6.94G/6.94G [07:22<00:00, 15.7MiB/s]

Модель 641087 успешно загружена и сохранена в /content/DeFooocus/models/checkpoints/realism.safetensors


In [ ]:
#@title DeFooocus (Лоры и тд)
#@markdown # ЛОРЫ/ЛИКОРИС/ЭМБЕДДИНГИ И ТД
#@markdown ### ID
model_ids = [219110, 635271, 584520, 141278, 661221]  # @param {type:"raw"}
#@markdown ### IMENA
namelora = ["Переславль", "Ретровейв", "Манга 1980е", "Фикс рук", "Бета нового Переславля"]  # @param {type:"raw"}

import requests
import os
from tqdm import tqdm
import http.client
from IPython.display import clear_output

api_key = "37886f12c162b410d63075416e9e0d00"
headers = {"Authorization": f"Bearer {api_key}"}

download_urls = [f'https://civitai.com/api/download/models/{model_id}' for model_id in model_ids]
save_paths = [f"/content/DeFooocus/models/loras/{name}.safetensors" for name in namelora]

for save_path in save_paths:
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

http.client.HTTPConnection._http_vsn = 11
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.1'

def download_model(url, path, name):
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        total_size = int(response.headers.get("content-length", 0))
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
                progress_bar.update(len(chunk))
        progress_bar.close()
        print(f"Модель {name} успешно загружена и сохранена в {path}")
    except requests.exceptions.RequestException as e:
        print(f"Произошла ошибка при загрузке модели {name}: {e}")
        if isinstance(e, http.client.IncompleteRead):
            partial_data = e.partial
            if response.raw._fp_bytes_read < total_size:
                remaining_data = response.raw.read(total_size - response.raw._fp_bytes_read)
                complete_data = partial_data + remaining_data
                with open(path, "ab") as f:
                    f.write(complete_data)
                print(f"Продолжили загрузку модели {name} и сохранили в {path}")

for url, path, name in zip(download_urls, save_paths, namelora):
    download_model(url, path, name)
    clear_output()

print(' [1;32mНу, тут совсем фарт')

 Ну, тут совсем фарт
